In [18]:
from collections import namedtuple

commands = open('./data.txt').read().splitlines()

def create_dir(name, parent):
    return {
        "name": name,
        "parent": parent,
        "subdirs": [],
        "files": [],
        "size": 0,
    }

def create_file(name, size):
    return { "name": name, "size": size }

def update_size(dir):
    fileSize = sum([file['size'] for file in dir['files']])
    dirSize = sum([update_size(subdir) for subdir in dir['subdirs']])

    dir['size'] = fileSize + dirSize
    return dir['size']

root = create_dir('/', None)

cur_dir = root

def run_command(cmd:str, cmds:iter):
    global root, cur_dir
    [_, part] = cmd.split(' ', 1)
    if part == 'ls':
        files = []
        file = next(cmds, None)
        while file and not file.startswith('$'):
            files.append(file)
            file = next(cmds, None)
        for x in files:
            [a, b] = x.split(' ')
            if a == 'dir':
                cur_dir['subdirs'].append(create_dir(b, cur_dir))
            else:
                cur_dir['files'].append(create_file(b, int(a)))

        # this one was not a file, it's the next command, so just run it
        if file:
            run_command(file, cmds)
    else:
        [_, new_dir] = part.split(' ')
        if new_dir == '/':
            cur_dir = root
        elif new_dir == '..':
            cur_dir = cur_dir['parent']
        else:
            cur_dir = next((dir for dir in cur_dir['subdirs'] if dir['name'] == new_dir))

def run_commands(lines: list[str]):
    cmds = iter(lines)

    while cmd := next(cmds, None):
        run_command(cmd, cmds)

def find_dirs(start, pred:callable):
    matches = []
    for dir in start['subdirs']:
        if pred(dir):
            matches.append(dir)
        matches.extend(find_dirs(dir, pred))
    return matches

run_commands(commands)

update_size(root)

matches = find_dirs(root, lambda dir: dir['size'] <= 100000)
part1_answer = sum([dir['size'] for dir in matches])

total_space = 70000000
needed_space = 30000000
max_usage = total_space - needed_space
to_delete = root['size'] - max_usage

big_enough = find_dirs(root, lambda dir: dir['size'] >= to_delete)
part2_answer = min([dir['size'] for dir in big_enough])

print(to_delete, part2_answer)

532950 545729
